In [1]:
#model套件
import torch
from torch import nn as nn
from torch.nn.parameter import Parameter as Parameter
from torch.utils.data import Dataset
from torch.autograd import Variable as Variable
from torchsummary import summary
import torch.nn.functional as F

#聲音處理套件
import librosa
#資料處理套件
import pandas as pd
import numpy as np
#時間套件
import time

from enum import Enum

In [2]:
#存檔位置
ModelMode = "LeNet5_ESC10_kFold"
TrainDataSoundMode = ["_1s", "_3s", "_5s", "_135s", "_531s"]
filePath_Train = ModelMode + TrainDataSoundMode[3] + "_EpochDown_Train.txt"
filePath_LastData = ModelMode + TrainDataSoundMode[3] +  "_EpochDown_LastData.txt"
modelPathHead = ModelMode + TrainDataSoundMode[3] + "_EpochDown"

In [3]:
#音訊資訊CSV檔位置
DATA_CSV_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/meta/esc50.csv"
#音訊存放位置
DATA_PATH = "D:/Hu_code/ESC-50-master/ESC-50-master/audio/"

def loadDataESC10(data_path, data_csv_path, soundLength):
    dataX_Fold = []
    dataY_Fold = []
    
    #讀取CSV檔
    df = pd.read_csv(data_csv_path)
    #篩選出ESC10作為訓練資料
    df_10 = df[df['esc10']==True]
    #取出類別，去除重複並排序
    classes = df_10['category'].unique()
    class_dict = {i:x for x,i in enumerate(classes)}
    #重新標label
    df_10['target'] = df_10['category'].map(class_dict)

    #計數
    count = 0
    
    for Fold in range(1, 6, 1):
        #初始化
        dataX = []
        dataY = []
        #篩選資料
        sound_df = df_10[df_10.fold == Fold]
        #轉換train資料/添加標籤
        for data in sound_df.itertuples():
            sig, sr = librosa.load(data_path+data[1])
            for i in range((6-soundLength)):
                sound = sig[i*sr : int((i+soundLength)*sr)]
                sound = audio_to_melspectrogram(audio=sound, sr=sr)
                dataX.append(sound)
                dataY.append(data[3])       
            count +=1
            print("{:.3f}".format(100 * count / len(df_10))+"%", end='\r')
        dataX_Fold.append(dataX)
        dataY_Fold.append(dataY)
                
    #通通轉成 torch 的 tensor 形式
    dataX_Fold = torch.Tensor(dataX_Fold)
    dataY_Fold = torch.Tensor(dataY_Fold).type(torch.LongTensor)
    
    #轉維度 FBCHW
    dataX_Fold = torch.reshape(dataX_Fold, (dataX_Fold.shape[0], dataX_Fold.shape[1], 1, dataX_Fold.shape[2], dataX_Fold.shape[3]))
           
    # 完成時打印最終消息
    print("Progress: 100% - Completed")
    return dataX_Fold, dataY_Fold

class conf:
    sr = 44100  #採樣率
    hop_length = 512  #帧移
    fmin = 20  #最低頻率
    fmax = sr // 2  #最高頻率
    n_mels = 128  #濾波器個數
    n_fft = 2048  #FFT窗口長度

#轉頻譜圖
def audio_to_melspectrogram(audio, sr):
    spectrogram = librosa.feature.melspectrogram(y=audio, 
                                                 sr=sr,
                                                 n_mels=128,
                                                 hop_length=512,
                                                 n_fft=2048,
                                                 fmin=20,
                                                 fmax=sr//2)
    spectrogram = librosa.power_to_db(spectrogram)
    return spectrogram

class CustomDatasetESC50(Dataset):
    def __init__(self, inputDataX, inputDataY, fold, isTrain):
        #防止範圍錯誤
        if(fold>5 or fold <1):
            raise ValueError("Fold範圍錯誤")
            
        #資料初始化
        self.dataX = []
        self.dataY = []
        
        # True:訓練資料  False:測試資料
        if isTrain:
            for Fold in range(1, 6, 1):
                if(Fold != fold):
                    for X in inputDataX[Fold-1]:
                        self.dataX.append(X)
                    for Y in inputDataY[Fold-1]:
                        self.dataY.append(Y)
        else:
            self.dataX = inputDataX[fold-1]
            self.dataY = inputDataY[fold-1]

    def __len__(self):
        return len(self.dataX)
    
    def __getitem__(self, index):
        # 根據索引返回單個樣本及其標籤
        data = self.dataX[index]
        label = self.dataY[index]
        
        return data, label

In [4]:
#LeNet5模型
class LeNet(nn.Module):
    def __init__(self, nums_class=10):
        super(LeNet,self).__init__()
        
        #卷基層
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=0),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        )
        
        #全連接層
        self.fullconn_layer = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(32,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,nums_class)           
        )
        
    def forward(self,x):
        output = self.conv_layer(x)
        output = self.fullconn_layer(output)
        return output

In [5]:
#訓練模型(全部訓練)
def train_model_TrainAll(trainFold, setLR, setEpoch):
    #用來記錄全部訓練過程的準確度和loss
    train_loss_array = []
    train_accuracy_array = []
    
    #---------------------------------------------------
    #訓練相關參數
    #---------------------------------------------------
    #訓練次數
    num_epochs = setEpoch
    #批次大小
    BATCH_SIZE = 64
    #學習率
    LR = setLR
    #loss方法
    loss_func = nn.CrossEntropyLoss()
    #優化器
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #scheduler
    #--------------------------------------------------------------------
    #調度器
    scheduler  = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)
    #--------------------------------------------------------------------
    
    #--------------------------------------------------------------------
    #資料抓入
    #預處理
    #--------------------------------------------------------------------
    #資料抓入
    trainloader = CustomDatasetESC50(inputDataX = dataX,
                               inputDataY = dataY,
                               fold = trainFold,
                               isTrain = True)
    #分批方法
    train_loader = torch.utils.data.DataLoader(trainloader, batch_size = BATCH_SIZE, shuffle = True)
    #--------------------------------------------------------------------
    
    #訓練次數
    for epoch in range(num_epochs):
        #訓練準確度
        correct_train = 0
        #訓練資料總筆數
        total_train = 0
        #批次訓練
        for i, data in enumerate(train_loader):
            #開啟模型訓練
            model.train()
            #將資料放入gpu
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            #清除梯度
            optimizer.zero_grad()
            #向前傳播
            outputs = model(inputs)
            #計算loss
            train_loss = loss_func(outputs, labels)
            #反向傳播
            train_loss.backward()
            #梯度下降
            optimizer.step()
            #找出信心度最高的判斷
            predicted = torch.max(outputs.data, 1)[1]
            #總筆數加上這次訓練的筆數
            total_train += len(labels)
            #計算判斷正確的筆數
            correct_train += (predicted == labels).float().sum()
        #計算這次訓練的準確度
        train_accuracy = 100 * correct_train / float(total_train)
        #紀錄這次訓練的準確度
        train_accuracy_array.append(train_accuracy)
        #紀錄這次訓練的loss
        train_loss_array.append(train_loss.data)
        
        #印出本次訓練結果
        print('Train Epoch: {}/{} Train_Loss: {} Train_acc: {:.6f}%'.format(epoch+1, num_epochs, train_loss.data, train_accuracy))
        #調度器走一步
        scheduler.step()
        
    #測試準確度
    test_accuracy1 = test_model(inputDataX = dataX1, inputDataY = dataY1, testFold = trainFold)
    test_accuracy3 = test_model(inputDataX = dataX3, inputDataY = dataY3, testFold = trainFold)
    test_accuracy5 = test_model(inputDataX = dataX5, inputDataY = dataY5, testFold = trainFold)
        
    #將gpu資料取回
    train_loss_array = torch.stack(train_loss_array,dim=-1).cpu()
    train_accuracy_array = torch.stack(train_accuracy_array,dim=-1).cpu()
    
    return train_loss_array, train_accuracy_array, test_accuracy1, test_accuracy3, test_accuracy5

In [6]:
#測試模型的準確度
def test_model(inputDataX, inputDataY, testFold):
    #固定模型不改變權重
    model.eval()
    #計數初始化
    correct = 0
    total = 0
    #資料抓取
    testloader = CustomDatasetESC50(inputDataX = inputDataX,
                               inputDataY = inputDataY,
                               fold = testFold,
                               isTrain = False)
    #封裝                                
    test_loader = torch.utils.data.DataLoader(testloader, batch_size=64, shuffle=False)
    
    #不計算梯度 加速計算
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            #預測結果
            outputs = model(inputs)
            
            #獲取預測標籤
            _, predicted = torch.max(outputs.data, 1)
            
            #計算準確度
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Test accuracy of the model: {:.6f}%'.format(100 * correct / total))
    return (100 * correct / total)

In [7]:
#紀錄開始時間
dataload_start_time = time.time()

#資料讀取
print("讀取1秒資料")
dataX1, dataY1 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 1)
print("讀取3秒資料")
dataX3, dataY3 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 3)
print("讀取5秒資料")
dataX5, dataY5 = loadDataESC10(data_path = DATA_PATH,
                            data_csv_path = DATA_CSV_PATH,
                            soundLength = 5)

#紀錄結束時間
dataload_end_time = time.time()

#計算耗時
dataload_training_time = dataload_end_time - dataload_start_time

#印出耗時
print("Data Loading Time: {:.2f} seconds".format(dataload_training_time))

讀取1秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_17948\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


C:\Users\test\AppData\Local\Temp\ipykernel_17948\2994481965.py:43: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  dataX_Fold = torch.Tensor(dataX_Fold)
C:\Users\test\AppData\Local\Temp\ipykernel_17948\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
讀取3秒資料
Progress: 100% - Completed
讀取5秒資料


C:\Users\test\AppData\Local\Temp\ipykernel_17948\2994481965.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10['target'] = df_10['category'].map(class_dict)


Progress: 100% - Completed
Data Loading Time: 28.64 seconds


In [8]:
kFoldTrainTime = []
kFoldTrainAccuracy = []
kFoldTestAccuracy1 = []
kFoldTestAccuracy3 = []
kFoldTestAccuracy5 = []

#設定存檔位置
f = open(filePath_Train, "w")
#k-fold訓練
for fold in range(1,6):
    kTLTrainTime = []
    kTLTrainAccuracy = []
    kTLTestAccuracy1 = []
    kTLTestAccuracy3 = []
    kTLTestAccuracy5 = []

    #當前裝置能使用的GPU數量
    num_devices = torch.cuda.device_count()
    print("可用的 GPU 数量:", num_devices)

    #建立模型
    model = LeNet(nums_class = 10)
    
    #將模型放置到 GPU 
    model = model.cuda()

    #使用 nn.DataParallel 包装模型
    #進行併行處理
    model = nn.DataParallel(model)
    
    #學習率
    lr=0.0032
    #epoch
    epoch=0
    for TL in range(3):
        print("----------------------------------------------------------------------------------------------------------------")
        print("fold: ", fold, "TL: ", TL)
        print("----------------------------------------------------------------------------------------------------------------")
        #訓練資料(要用幾秒訓練)
        if(TL==0):
            #1s
            dataX = dataX1
            dataY = dataY1
            epoch = 60
        elif(TL==1):
            #3s
            dataX = dataX3
            dataY = dataY3
            epoch = 40
        else:
            #5s
            dataX = dataX5
            dataY = dataY5
            epoch = 20

        #紀錄訓練開始時間
        start_time = time.time()

        #訓練模型(只訓練分類器)
        train_loss, train_accuracy, test_accuracy1, test_accuracy3, test_accuracy5 = train_model_TrainAll(trainFold=fold, setLR=lr, setEpoch=epoch)

        #紀錄訓練結束時間
        end_time = time.time()

        #計算訓練耗時
        training_time = end_time - start_time

        #印出訓練耗時
        print("Training Time: {:.2f} seconds".format(training_time))

        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("訓練過程\n")
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        for i in range(len(train_loss)):
            # 寫入資料
            f.write("Fold: {} TL: {} Train Epoch: {}/{} Train_Loss: {} Train_Acc: {:.6f}%\n".format(fold, TL, i+1, len(train_loss), train_loss[i], train_accuracy[i]))
        f.write("---------------------------------------------------------------------------------------------------------------\n")
        f.write("Test_Accuracy1: {:.6f}% Test_Accuracy3: {:.6f}% Test_Accuracy5: {:.6f}%\n".format(test_accuracy1, test_accuracy3, test_accuracy5))
        f.write("Training Time: {:.2f} seconds\n".format(training_time))
        f.write("---------------------------------------------------------------------------------------------------------------\n")

        #--------------------------------------------------------------------
        
        kTLTrainTime.append(training_time)
        kTLTrainAccuracy.append(train_accuracy[-1])
        kTLTestAccuracy1.append(test_accuracy1)
        kTLTestAccuracy3.append(test_accuracy3)
        kTLTestAccuracy5.append(test_accuracy5)
        
    kFoldTrainTime.append(kTLTrainTime)
    kFoldTrainAccuracy.append(kTLTrainAccuracy)
    kFoldTestAccuracy1.append(kTLTestAccuracy1)
    kFoldTestAccuracy3.append(kTLTestAccuracy3)
    kFoldTestAccuracy5.append(kTLTestAccuracy5)
    
    modelpath = modelPathHead + "_" + str(fold) + "fold.pth"
    #保存模型
    torch.save(model.state_dict(), modelpath)
    
#關閉
f.close()

可用的 GPU 数量: 1
----------------------------------------------------------------------------------------------------------------
fold:  1 TL:  0
----------------------------------------------------------------------------------------------------------------
Train Epoch: 1/60 Train_Loss: 2.047257900238037 Train_acc: 19.875000%
Train Epoch: 2/60 Train_Loss: 1.9943618774414062 Train_acc: 23.312500%
Train Epoch: 3/60 Train_Loss: 1.6331409215927124 Train_acc: 31.625000%
Train Epoch: 4/60 Train_Loss: 1.7535151243209839 Train_acc: 33.187500%
Train Epoch: 5/60 Train_Loss: 1.2133461236953735 Train_acc: 39.687500%
Train Epoch: 6/60 Train_Loss: 1.4670765399932861 Train_acc: 40.750000%
Train Epoch: 7/60 Train_Loss: 1.3427127599716187 Train_acc: 45.562500%
Train Epoch: 8/60 Train_Loss: 1.133111834526062 Train_acc: 49.000000%
Train Epoch: 9/60 Train_Loss: 1.2241837978363037 Train_acc: 51.625000%
Train Epoch: 10/60 Train_Loss: 1.1607385873794556 Train_acc: 51.000000%
Train Epoch: 11/60 Train_Loss: 1.88

Train Epoch: 1/20 Train_Loss: 2.150801658630371 Train_acc: 50.937500%
Train Epoch: 2/20 Train_Loss: 1.0187993049621582 Train_acc: 65.312500%
Train Epoch: 3/20 Train_Loss: 0.8516100645065308 Train_acc: 65.937500%
Train Epoch: 4/20 Train_Loss: 0.7657455205917358 Train_acc: 70.312500%
Train Epoch: 5/20 Train_Loss: 0.6228582262992859 Train_acc: 71.250000%
Train Epoch: 6/20 Train_Loss: 0.7493678331375122 Train_acc: 73.750000%
Train Epoch: 7/20 Train_Loss: 0.7560524344444275 Train_acc: 71.875000%
Train Epoch: 8/20 Train_Loss: 0.4966738820075989 Train_acc: 79.687500%
Train Epoch: 9/20 Train_Loss: 0.39423951506614685 Train_acc: 80.937500%
Train Epoch: 10/20 Train_Loss: 0.402726411819458 Train_acc: 85.937500%
Train Epoch: 11/20 Train_Loss: 0.8162370324134827 Train_acc: 81.875000%
Train Epoch: 12/20 Train_Loss: 0.7017034888267517 Train_acc: 76.875000%
Train Epoch: 13/20 Train_Loss: 0.5509731769561768 Train_acc: 77.500000%
Train Epoch: 14/20 Train_Loss: 0.7179716229438782 Train_acc: 76.250000%
Tr

Train Epoch: 26/40 Train_Loss: 0.8516364693641663 Train_acc: 69.479172%
Train Epoch: 27/40 Train_Loss: 0.8198773264884949 Train_acc: 66.770836%
Train Epoch: 28/40 Train_Loss: 0.7145596146583557 Train_acc: 69.270836%
Train Epoch: 29/40 Train_Loss: 0.8851615190505981 Train_acc: 69.687500%
Train Epoch: 30/40 Train_Loss: 0.5496501326560974 Train_acc: 71.875000%
Train Epoch: 31/40 Train_Loss: 0.5967651605606079 Train_acc: 75.416672%
Train Epoch: 32/40 Train_Loss: 0.6892462968826294 Train_acc: 75.104172%
Train Epoch: 33/40 Train_Loss: 0.61017906665802 Train_acc: 74.583336%
Train Epoch: 34/40 Train_Loss: 0.5563097596168518 Train_acc: 75.520836%
Train Epoch: 35/40 Train_Loss: 0.6886175870895386 Train_acc: 74.687508%
Train Epoch: 36/40 Train_Loss: 0.6824191808700562 Train_acc: 75.416672%
Train Epoch: 37/40 Train_Loss: 0.5462997555732727 Train_acc: 76.041672%
Train Epoch: 38/40 Train_Loss: 0.6899011731147766 Train_acc: 78.541672%
Train Epoch: 39/40 Train_Loss: 0.6349207758903503 Train_acc: 77.50

Train Epoch: 4/40 Train_Loss: 0.44685110449790955 Train_acc: 77.500008%
Train Epoch: 5/40 Train_Loss: 0.6073650121688843 Train_acc: 78.958336%
Train Epoch: 6/40 Train_Loss: 0.4929978549480438 Train_acc: 78.541672%
Train Epoch: 7/40 Train_Loss: 0.4732970595359802 Train_acc: 80.625008%
Train Epoch: 8/40 Train_Loss: 0.6415401697158813 Train_acc: 76.770836%
Train Epoch: 9/40 Train_Loss: 0.43358975648880005 Train_acc: 80.520836%
Train Epoch: 10/40 Train_Loss: 0.4275483787059784 Train_acc: 83.125008%
Train Epoch: 11/40 Train_Loss: 0.49788379669189453 Train_acc: 79.375008%
Train Epoch: 12/40 Train_Loss: 0.3358030319213867 Train_acc: 79.375008%
Train Epoch: 13/40 Train_Loss: 0.3332972526550293 Train_acc: 79.895836%
Train Epoch: 14/40 Train_Loss: 0.4525890052318573 Train_acc: 83.229172%
Train Epoch: 15/40 Train_Loss: 0.4664456844329834 Train_acc: 82.291672%
Train Epoch: 16/40 Train_Loss: 0.1915443390607834 Train_acc: 80.937508%
Train Epoch: 17/40 Train_Loss: 0.3551759421825409 Train_acc: 85.312

Train Epoch: 47/60 Train_Loss: 0.5046772360801697 Train_acc: 82.312500%
Train Epoch: 48/60 Train_Loss: 0.44461870193481445 Train_acc: 82.937500%
Train Epoch: 49/60 Train_Loss: 0.2967431843280792 Train_acc: 82.125000%
Train Epoch: 50/60 Train_Loss: 0.39312806725502014 Train_acc: 82.500000%
Train Epoch: 51/60 Train_Loss: 0.4092346727848053 Train_acc: 84.250000%
Train Epoch: 52/60 Train_Loss: 0.4604514241218567 Train_acc: 83.937500%
Train Epoch: 53/60 Train_Loss: 0.2854577600955963 Train_acc: 83.375000%
Train Epoch: 54/60 Train_Loss: 0.3712429404258728 Train_acc: 84.312500%
Train Epoch: 55/60 Train_Loss: 0.27946993708610535 Train_acc: 84.875000%
Train Epoch: 56/60 Train_Loss: 0.29087033867836 Train_acc: 84.062500%
Train Epoch: 57/60 Train_Loss: 0.4788743555545807 Train_acc: 83.437500%
Train Epoch: 58/60 Train_Loss: 0.6395518779754639 Train_acc: 84.000000%
Train Epoch: 59/60 Train_Loss: 0.3070490062236786 Train_acc: 84.500000%
Train Epoch: 60/60 Train_Loss: 0.4017254114151001 Train_acc: 84

Train Epoch: 25/60 Train_Loss: 0.8032475709915161 Train_acc: 72.812500%
Train Epoch: 26/60 Train_Loss: 0.7666130065917969 Train_acc: 72.000000%
Train Epoch: 27/60 Train_Loss: 0.8110519051551819 Train_acc: 68.375000%
Train Epoch: 28/60 Train_Loss: 0.7918573021888733 Train_acc: 70.312500%
Train Epoch: 29/60 Train_Loss: 0.5163389444351196 Train_acc: 74.437500%
Train Epoch: 30/60 Train_Loss: 0.536405086517334 Train_acc: 75.625000%
Train Epoch: 31/60 Train_Loss: 0.7254565358161926 Train_acc: 77.750000%
Train Epoch: 32/60 Train_Loss: 0.5459692478179932 Train_acc: 79.000000%
Train Epoch: 33/60 Train_Loss: 0.5568895936012268 Train_acc: 78.687500%
Train Epoch: 34/60 Train_Loss: 0.5291915535926819 Train_acc: 80.312500%
Train Epoch: 35/60 Train_Loss: 0.46457067131996155 Train_acc: 80.437500%
Train Epoch: 36/60 Train_Loss: 0.7686435580253601 Train_acc: 79.562500%
Train Epoch: 37/60 Train_Loss: 0.6108744144439697 Train_acc: 80.250000%
Train Epoch: 38/60 Train_Loss: 0.7663268446922302 Train_acc: 80.

In [9]:
class FilterMode(Enum):
    Max = "Max"
    Min = "Min"
    Mean = "Mean"
    
def listFilter(data, mode:FilterMode):
    TL = []
    for fold in range(len(data)):
        TL.append(data[fold][len(data[fold])-1])
    
    switch = {
        FilterMode.Max: max(TL),
        FilterMode.Min: min(TL),
        FilterMode.Mean: np.mean(TL)
    }

    return switch.get(mode, -1)

#設定存檔位置
f = open(filePath_LastData, "w")

#整理出訓練完的數值
f.write("---------------------------------------------------------------------------------------------------------------\n")
f.write("訓練結果\n")
f.write("-----------------------------------------------------------------------------------\n")
for fold in range(5):
    for TL in range(3):
        # 寫入資料
        f.write("Fold: {} TL: {} Train_acc: {:.10f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}% Training Time: {:.2f} seconds\n".format(fold+1, TL, kFoldTrainAccuracy[fold][TL], kFoldTestAccuracy1[fold][TL], kFoldTestAccuracy3[fold][TL], kFoldTestAccuracy5[fold][TL], kFoldTrainTime[fold][TL]))
    f.write("-----------------------------------------------------------------------------------\n")
    
# 寫入資料[]
f.write("Max Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Max), listFilter(kFoldTestAccuracy1, FilterMode.Max), listFilter(kFoldTestAccuracy3, FilterMode.Max), listFilter(kFoldTestAccuracy5, FilterMode.Max)))
f.write("Min Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Min), listFilter(kFoldTestAccuracy1, FilterMode.Min), listFilter(kFoldTestAccuracy3, FilterMode.Min), listFilter(kFoldTestAccuracy5, FilterMode.Min)))
f.write("Avg Train_acc: {:.6f}% Test_accuracy1: {:.6f}% Test_accuracy3: {:.6f}% Test_accuracy5: {:.6f}%\n".format(listFilter(kFoldTrainAccuracy, FilterMode.Mean), listFilter(kFoldTestAccuracy1, FilterMode.Mean), listFilter(kFoldTestAccuracy3, FilterMode.Mean), listFilter(kFoldTestAccuracy5, FilterMode.Mean)))

f.close()